In [2]:
import torch
from safetensors.torch import load_file
from transformers import RobertaConfig, RobertaForSequenceClassification

# 모델 경로 설정
model_path = "C:/fintech_service/final_project/saved_model_KLUE-RoBERTa/saved_model"  # 절대 경로로 지정

# config.json 로드
config = RobertaConfig.from_pretrained(model_path)

# 정확한 safetensors 파일 경로
safetensors_path = f"{model_path}/model.safetensors"
state_dict = load_file(safetensors_path)

# 모델 생성 및 가중치 적용
model = RobertaForSequenceClassification(config)
model.load_state_dict(state_dict)

# pytorch_model.bin으로 저장
pytorch_model_path = f"{model_path}/pytorch_model.bin"
torch.save(model.state_dict(), pytorch_model_path)

print(f"모델이 {pytorch_model_path}에 저장되었습니다.")


모델이 C:/fintech_service/final_project/saved_model_KLUE-RoBERTa/saved_model/pytorch_model.bin에 저장되었습니다.


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 모델과 토크나이저 로드
model_path = "C:/fintech_service/final_project/saved_model_KLUE-RoBERTa/saved_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # 모델을 평가 모드로 설정

print("모델이 성공적으로 로드되었습니다!")


모델이 성공적으로 로드되었습니다!


In [4]:
import os
import zipfile
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 1. 압축 해제
zip_path = 'C:/fintech_service/final_project/saved_model.zip'
extract_path = 'C:/fintech_service/final_project/saved_model'

if not os.path.exists(extract_path):  # 이미 압축이 풀려있는지 확인
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# 2. 모델과 토크나이저 로드
model_path = 'C:/fintech_service/final_project/saved_model_KLUE-RoBERTa/saved_model'  # 정확한 경로로 수정
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # 모델을 평가 모드로 설정

# 3. 감성 분석할 CSV 파일 로드
input_file = "C:/fintech_service/final_project/data/뱅뱅막국수_combined_reviews_감정분석완료_재분석.csv"  # 분석할 파일 경로
df = pd.read_csv(input_file)

# 문자열이 아닌 값(NaN 등)을 제거
df = df.dropna(subset=['Review'])

# 4. 감성 분석 함수 정의
def predict_sentiment(text):
    if not isinstance(text, str):
        return "알 수 없음"  # 문자열이 아닌 경우 기본값 반환
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "긍정" if prediction == 1 else "부정"

# 5. 부정적으로 판별된 리뷰에 대해서만 재분석 수행
def reanalyze_if_negative(sentiment, text):
    if sentiment == "부정":
        # 부정적으로 판별된 경우에만 재분석 수행
        second_prediction = predict_sentiment(text)
        return second_prediction
    # 부정적이지 않은 경우 원래 값을 그대로 유지
    return sentiment

# 'sentiment' 컬럼이 부정적인 경우에만 재분석 수행하여 'Sentiment2'에 저장
df['Sentiment2'] = df.apply(lambda row: reanalyze_if_negative(row['sentiment'], row['Review']), axis=1)

# 6. 결과 파일 저장
base_name = os.path.splitext(os.path.basename(input_file))[0]  # 파일명만 추출 (확장자 제외)
output_dir = os.path.dirname(input_file)  # 기존 파일이 있는 폴더
output_file = os.path.join(output_dir, f"{base_name}_재분석_RoBERTa.csv")

df.to_csv(output_file, index=False)
print(f"감성 분석 결과가 {output_file}에 저장되었습니다.")


감성 분석 결과가 C:/fintech_service/final_project/data\뱅뱅막국수_combined_reviews_감정분석완료_재분석_재분석_RoBERTa.csv에 저장되었습니다.
